In [4]:
#pip install --upgrade skillcorner

  Using cached skillcorner-2.6.11-py3-none-any.whl.metadata (4.2 kB)
  Using cached fitrequest-1.4.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached makefun-1.16.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached progress-1.6.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached aiofiles-25.1.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached ClusterShell-1.9.3.tar.gz (417 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jsonpath_ng-1.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached StrEnum-0.4.15-py3-none-any.whl.metadata (5.3 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.me

  DEPRECATION: Building 'clustershell' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'clustershell'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppre

In [37]:
from skillcorner.client import SkillcornerClient
import pandas as pd
import json
import numpy as np
import math
import os
import re

def load_match_ids_from_folder(folder="."):
    """
    Automatically extract numeric match_id from .csv filenames in the folder
    e.g., 'Counterattack_player_data_1585551.csv' → 1585551
    """
    match_ids = []

    for fname in os.listdir(folder):
        if fname.lower().endswith(".csv"):   # Only CSV files
            m = re.search(r"(\d+)", fname)   # Extract numeric part from filename
            if m:
                match_ids.append(int(m.group(1)))

    match_ids = sorted(set(match_ids))       # Deduplicate and sort
    return match_ids
    
client = SkillcornerClient(username = "jcd0119@auburn.edu",password = "2026_ASI_Summit!")

def load_match(match_id):
    print(f"📥 Loading match {match_id}")
    return client.get_match_tracking_data(match_id=match_id, params={'competition edition': 800})

def build_player_data(raw):
    df = pd.DataFrame(raw)

    frames_expanded = df.explode("player_data", ignore_index=True)
    frames_expanded = frames_expanded.dropna(subset=["player_data"])
    frames_expanded = frames_expanded[
        frames_expanded["player_data"].apply(lambda v: isinstance(v, dict))
    ].reset_index(drop=True)

    player_data = pd.concat([
        frames_expanded.drop(columns=["player_data"]),
        pd.json_normalize(frames_expanded["player_data"])
    ], axis=1)

    # normalize ball & possession
    for col in ["ball_data", "possession"]:
        if col in frames_expanded.columns:
            safe = frames_expanded[col].apply(lambda v: v if isinstance(v, dict) else {})
            normalized = pd.json_normalize(safe)
            normalized.columns = (f"{col}.{c}" for c in normalized.columns)
            player_data = pd.concat([player_data.drop(columns=[col]), normalized], axis=1)

    # drop image corners
    if "image_corners_projection" in player_data.columns:
        player_data = player_data.drop(columns=["image_corners_projection"])

    return player_data

def build_team_map(match_tracking_data):
    team_map = {}

    # Extract team info from possession
    for frame in match_tracking_data:
        poss = frame.get("possession", {})
        pid  = poss.get("player_id", None)
        grp  = poss.get("group", None)

        if pid is not None and grp is not None:
            team_map[pid] = grp   # "home team", "away team"

    return team_map

def assign_team_side(player_data, team_map):

    player_data["team_side"] = player_data["player_id"].map(
        lambda pid: team_map.get(pid, None)
    )

    return player_data

def detect_home_left_first_half(player_data):

    first_frame = player_data["frame"].min()
    first_pos   = player_data[player_data["frame"] == first_frame]

    home_x = first_pos[first_pos["team_side"] == "home team"]["x"].median()
    away_x = first_pos[first_pos["team_side"] == "away team"]["x"].median()

    home_left_first_half = (home_x < away_x)
    return home_left_first_half

def get_team_halves(team, period, home_left_first_half):
    """
    team: 'home team' / 'away team'
    return: (own_sign, opp_sign)
      own_sign: team's own goal direction (-1: left, +1: right)
      opp_sign: opponent goal direction
    """
    if period == 1:  # First half
        if home_left_first_half:
            home_own = -1   # Home goal = left
            away_own = +1   # Away goal = right
        else:
            home_own = +1
            away_own = -1
    else:            # Second half (switched sides)
        if home_left_first_half:
            home_own = +1   # Home goal = right
            away_own = -1   # Away goal = left
        else:
            home_own = -1
            away_own = +1

    if team == "home team":
        own = home_own
    else:
        own = away_own

    opp = -own
    return own, opp
    
def detect_counterattack(player_data, home_left_first_half):

    ball_x = player_data.groupby("frame")["ball_data.x"].first()
    possession = player_data.groupby("frame")["possession.group"].first()
    period = player_data.groupby("frame")["period"].first()

    home_df = player_data[player_data["team_side"] == "home team"]
    away_df = player_data[player_data["team_side"] == "away team"]

    home_by_frame = home_df.groupby("frame")
    away_by_frame = away_df.groupby("frame")

    counterattack = {}

    for f in ball_x.index:

        poss = possession[f]
        bx = ball_x[f]
        per = period[f]

        if poss not in ["home team", "away team"]:
            counterattack[f] = False
            continue

        own_sign, opp_sign = get_team_halves(poss, per, home_left_first_half)

        # Attack only when the ball is in the opponent half
        if np.sign(bx) != opp_sign:
            counterattack[f] = False
            continue

        # Get opponent defenders
        opp_group = away_by_frame if poss == "home team" else home_by_frame
        try:
            opp_players = opp_group.get_group(f)
        except:
            counterattack[f] = False
            continue

        if opp_sign == +1:
            num_def = (opp_players["x"] > bx).sum()
        else:
            num_def = (opp_players["x"] < bx).sum()

        counterattack[f] = (num_def <= 4)

    return counterattack

def make_plays(frames, block_size=100, max_frame=None):

    frames = sorted(frames)
    plays = []

    start = frames[0]

    while True:
        end = start + block_size

        if end > max_frame:
            break

        plays.append((start, end))

        next_candidates = [f for f in frames if f > end]

        if not next_candidates:
            break

        start = min(next_candidates)

    return plays
    
def extract_play_player_data(player_data, play):
    start, end = play
    return player_data[(player_data["frame"] >= start) & (player_data["frame"] <= end)].copy()

def extract_all_plays(player_data, plays):
    play_dfs = []
    for (s, e) in plays:
        df_play = extract_play_player_data(player_data, (s, e))
        play_dfs.append(df_play)
    return play_dfs

def combine_plays(play_dfs):
    out = []
    for i, df in enumerate(play_dfs):
        temp = df.copy()
        temp["play_id"] = i
        out.append(temp)
    return pd.concat(out, ignore_index=True)
    
def normalize_direction_by_ball(plays_combined_df):
    df = plays_combined_df.copy()

    # Columns to store normalized results
    df["flip"]         = np.nan
    df["x_norm"]       = np.nan
    df["y_norm"]       = np.nan
    df["ball_x_norm"]  = np.nan
    df["ball_y_norm"]  = np.nan

    # Unique play IDs
    play_ids = df["play_id"].unique()

    for pid in play_ids:
        play = df[df["play_id"] == pid]

        # First frame of the play
        f0 = play["frame"].min()

        # Ball location at starting frame
        ball_x0 = play.loc[play["frame"] == f0, "ball_data.x"].iloc[0]

        # Determine flip direction
        flip = -1 if ball_x0 < 0 else 1

        # Apply to all rows in the play
        idx = (df["play_id"] == pid)

        df.loc[idx, "flip"]        = flip
        df.loc[idx, "x_norm"]      = df.loc[idx, "x"] * flip
        df.loc[idx, "y_norm"]      = df.loc[idx, "y"]
        df.loc[idx, "ball_x_norm"] = df.loc[idx, "ball_data.x"] * flip
        df.loc[idx, "ball_y_norm"] = df.loc[idx, "ball_data.y"]

    return df

def save_csv(normalized_df, match_id):
    fname = f"Counterattack_{match_id}.csv"
    normalized_df.to_csv(fname, index=False)
    print(f"Saved: {fname}")

def run_match_pipeline(match_id):

    print("\n==============================")
    print(f"🚀 Processing match {match_id}")
    print("==============================\n")

    # 1) Load raw data
    raw = load_match(match_id)

    # 2) Build player_data from raw
    player_data = build_player_data(raw)

    # 3) Build team map
    team_map = build_team_map(raw)

    # 4) Assign team side (home/away)
    player_data = assign_team_side(player_data, team_map)

    # 5) Detect if home team starts on the left side
    home_left_first_half = detect_home_left_first_half(player_data)
    print(f"home_left_first_half = {home_left_first_half}")

    # 6) Detect counterattack frames
    counterattack = detect_counterattack(player_data, home_left_first_half)
    counterattack_frames = sorted([f for f, v in counterattack.items() if v])
    print(f"Total counterattack frames: {len(counterattack_frames)}")

    if len(counterattack_frames) == 0:
        print("❌ No counterattack frames found. Skipping match.")
        return None

    # 7) Build plays
    max_frame = player_data["frame"].max()
    plays = make_plays(counterattack_frames, block_size=100, max_frame=max_frame)
    print(f"Total plays extracted: {len(plays)}")

    if len(plays) == 0:
        print("❌ No valid plays (100 frames) found. Skipping match.")
        return None

    # 8) Extract each play's player data
    all_play_dfs = extract_all_plays(player_data, plays)

    # 9) Combine into a single DataFrame
    plays_combined_df = combine_plays(all_play_dfs)

    # 10) Normalize direction using ball direction
    normalized_df = normalize_direction_by_ball(plays_combined_df)

    # 11) Save to CSV
    save_csv(normalized_df, match_id)

    print(f"\n🎉 Match {match_id} processing complete!\n")

    return normalized_df


In [39]:
match_ids = load_match_ids_from_folder(".")

total = len(match_ids)
print(f"총 {total}개의 match_id가 감지되었습니다.\n")

for idx, mid in enumerate(match_ids, start=1):
    print(f"➡ Processing {idx}/{total} | match_id = {mid}")
    run_match_pipeline(mid)

총 99개의 match_id가 감지되었습니다.

➡ Processing 1/99 | match_id = 1585551

🚀 Processing match 1585551

📥 Loading match 1585551
home_left_first_half = False
Total counterattack frames: 4171
Total plays extracted: 149
Saved: Counterattack_1585551.csv

🎉 Match 1585551 processing complete!

➡ Processing 2/99 | match_id = 1587604

🚀 Processing match 1587604

📥 Loading match 1587604
home_left_first_half = False
Total counterattack frames: 4409
Total plays extracted: 147
Saved: Counterattack_1587604.csv

🎉 Match 1587604 processing complete!

➡ Processing 3/99 | match_id = 1588669

🚀 Processing match 1588669

📥 Loading match 1588669
home_left_first_half = True
Total counterattack frames: 4150
Total plays extracted: 158
Saved: Counterattack_1588669.csv

🎉 Match 1588669 processing complete!

➡ Processing 4/99 | match_id = 1588878

🚀 Processing match 1588878

📥 Loading match 1588878
home_left_first_half = True
Total counterattack frames: 3565
Total plays extracted: 134
Saved: Counterattack_1588878.csv

🎉